In [ ]:
# Import libraries
from Tracker import Tracker

import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
tracker_obj = Tracker()
tracker_obj.update_time_calculations()

total_covered = tracker_obj.get_total_time_covered()
remaining_weekly = tracker_obj.get_remaining_weekly()

In [ ]:
days_df = pd.DataFrame(tracker_obj.get_days_stats())
daily_max_minutes = tracker_obj.get_max_minutes_daily()
avg_minutes, avg_str_display = tracker_obj.find_average_time_to_cover()

fig = go.Figure(go.Bar(
            x=days_df['minutes_covered'],
            y=days_df['day'],
            orientation='h',
            name="",
            customdata=days_df['coverage'],
            hovertemplate = "Total: %{x}<br>Coverage: %{customdata}"
))

fig.add_shape(
        dict(
            type="line",
            x0=daily_max_minutes,
            x1=daily_max_minutes,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="Red",
                width=2,
                dash="dot"
            )
))

# Average time to cover
fig.add_shape(
        dict(
            type="line",
            x0=avg_minutes,
            x1=avg_minutes,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="#FECB52",
                width=2,
                dash="dot"
            ),
))

fig.update_layout(
    title_text='Weekly hours calculation (Individual Days)',
    xaxis_title='Minutes covered',
    yaxis_title='Day',
    yaxis=dict(autorange="reversed"),
    width=1000,
    height=600,
    xaxis = dict(
      range=[0,600],
        tick0 = 0,
        dtick = 60
    )
)

fig.show()

In [ ]:
time_analysis_dict = {
    'before_noon': tracker_obj.get_before_noon_minutes(),
    'after_noon': tracker_obj.get_after_noon_minutes()
}

time_analysis_df = pd.DataFrame(time_analysis_dict.items(), columns=['category', 'minutes'])

fig = go.Figure()

fig.add_trace(go.Bar(
            name = "",
            x=time_analysis_df['category'],
            y=time_analysis_df['minutes'],
            hovertemplate = "Total(minutes): %{y}"
        )
)

# Hours should be 50-50 coverage
fig.add_shape(
        dict(
            type="line",
            x0=-0.5,
            x1=1.5,
            y0=1200, # Half of 40 hours is 20 hours (1200 minutes)
            y1=1200,
            line=dict(
                color="#FECB52",
                width=2,
                dash="dot"
            ),
))

fig.update_layout(
    title_text='Comparisons: Before 12pm vs After 12pm',
    yaxis_title='Minutes covered',
    width=800,
    height=800,
    yaxis = dict(
      range=[0,2000],
        tick0 = 0,
        dtick = 250
    ),
    xaxis = dict(
        tickmode = 'array',
        tickvals = time_analysis_df['category'],
        ticktext=['Before 12pm','After 12pm']
    )
)

fig.show()

In [ ]:
covered_today, balance_today = tracker_obj.find_time_covered_today()

buffer_name = ''
buffer_color = ''

if balance_today > 0:
    buffer_name = 'remaining'
    buffer_color = 'rgba(237, 49, 49, 0.6)'
else:
    buffer_name = 'overtime'
    buffer_color = 'rgba(49, 237, 86, 0.6)'
    balance_today = abs(balance_today)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=['Today'],
    x=[covered_today],
    name='covered',
    orientation='h',
    hovertemplate = "Total(minutes): %{x} </br>",
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
    )
))

fig.add_trace(go.Bar(
    y=['Today'],
    x=[balance_today],
    name=buffer_name,
    orientation='h',
    hovertemplate = "Total(minutes): %{x} </br>",
    marker=dict(
        color=buffer_color,
    )
))

fig.update_layout(
    barmode='stack',
    autosize=False,
    width=800,
    height=300
)

fig.update_yaxes(automargin=True)

fig.show()

In [ ]:
# Pie chart
total_calculation_df = pd.DataFrame([
    {
        'category': 'covered', 
        'amount': total_covered, 
        'amount_hrs': tracker_obj.get_hours_minutes(total_covered)
    },
    {
        'category': 'remaining', 
        'amount': remaining_weekly,
        'amount_hrs': tracker_obj.get_hours_minutes(remaining_weekly)
    }
])

fig = go.Figure(go.Pie(
    name="",
    values = total_calculation_df['amount'],
    labels = total_calculation_df['category'],
    customdata = total_calculation_df['amount_hrs'],
    hovertemplate = "Category: %{label} <br>Total(minutes): %{value} </br>Total(hours): %{customdata}"

))

fig.update_layout(title_text='Overall weekly hours calculation: Remaining vs Covered')

fig.show()